In [1]:
# Approach I found in the history of the only netCDF file in the b10-simulation
# Thu Apr 02 03:49:15 2020: cdo mergetime /work/mh1119/DYAMOND_winter/sst_sic_final.nc* /work/mh1119/DYAMOND_winter/sst-sic-runmean_0017_R02B10.nc
# Thu Apr 02 01:24:21 2020: cdo setmisstonn /work/mh1119/DYAMOND_winter/sst_sic_missing.nc.2020-01-13 /work/mh1119/DYAMOND_winter/sst_sic_final.nc2020-01-13
# Thu Apr 02 01:01:48 2020: cdo seldate,2020-01-13 /work/mh1119/DYAMOND_winter/sst_sic_missing.nc /work/mh1119/DYAMOND_winter/sst_sic_missing.nc.2020-01-13
# Thu Apr 02 00:55:45 2020: cdo -P 8 -f nc4 remapnn,/home/dkrz/k203123/experiments/input/2.5km/icon_grid_0017_R02B10_G.nc -setpartabn,grib2cf.tab /work/mh1119/DYAMOND_winter/dyamond-winter-sst-sic-runmean.grb /work/mh1119/DYAMOND_winter/sst_sic_missing.nc

# Approach which works as well
# cdo -P 8 -f nc4 selvar,tp -sellonlatbox,-180,180,-20,20 -setgrid,/pool/data/ICON/grids/public/mpim/0017/icon_grid_0017_R02B10_G.nc /work/bk1040/experiments/DYAMOND_winter/nwp_R2B10_lkm1007_atm2_2d_ml_20200130T000000Z.grb tp_20200130_tropics.nc

In [2]:
from getpass import getuser # Libary to copy things
from pathlib import Path    # Object oriented libary to deal with paths
import os
from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
from subprocess import run, PIPE
import sys
 
import dask # Distributed data libary
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
import xarray as xr # Libary to work with labeled n-dimensional data and dask

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Set some user specific variables
scratch_dir = Path('/scratch') / getuser()[0] / getuser() # Define the users scratch dir

# Create a temp directory where the output of distributed cluster will be written to, after this notebook
# is closed the temp directory will be closed
dask_tmp_dir = TemporaryDirectory(dir=scratch_dir, prefix='ml2pl_')

cluster = SLURMCluster(memory='500GiB',
                       cores=72,
                       project='mh0731',
                       walltime='2:00:00',
                       queue='gpu',
                       name='ml2pl',
                       scheduler_options={'dashboard_address': ':12435'},
                       local_directory=dask_tmp_dir.name,
                       job_extra=[#f'--nodelist=m21578',
                                  f'-J ml2pl', 
                                  f'-D {dask_tmp_dir.name}',
                                  f'--begin=now',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o',
                                  f'--output={dask_tmp_dir.name}/LOG_cluster.%j.o'
                                 ],
                       interface='ib0')

cluster.scale(jobs=1)
dask_client = Client(cluster)
dask_client.wait_for_workers(2)

In [ ]:
data_path = Path('/work/bk1040/experiments/DYAMOND_winter/')
glob_pattern = 'atm_3d_u_ml'
# if dont take first element(s), theres a subdir with more matching files, we dont want that
wind_files = sorted([str(f) for f in data_path.rglob(f'*{glob_pattern}*.grb')])[2:]
glob_pattern = 'atm_3d_pres_ml'
pres_files = sorted([str(f) for f in data_path.rglob(f'*{glob_pattern}*.grb')])[2:]

In [ ]:
wind_files[40]

In [ ]:
@dask.delayed
def run_cmd(cmd, path_extra=Path(sys.exec_prefix)/'bin'):
    '''Run a bash command.'''
    env_extra = os.environ.copy()
    env_extra['PATH'] = str(path_extra) + ':' + env_extra['PATH']
    
    status = run(cmd, check=False, stderr=PIPE, stdout=PIPE, env=env_extra)
    
    if status.returncode != 0:
        error = f'''{' '.join(cmd)}: {status.stderr.decode('utf-8')}'''
        raise RuntimeError(f'{error}')
    return status.stdout.decode('utf-8')

In [ ]:
home = Path(os.path.expanduser('~'))
work_dir = Path('/work/mh0731/m300414/DyWinter_b10/')
name_table = work_dir / 'mypartab.txt'

In [ ]:
run_futures = []
for windfile, presfile in zip(wind_files[38:40], pres_files[38:40]):  
    
    print(windfile[-20:-7])
    date = windfile[-20:-7]

    onelevelfile = work_dir / 'U_Wind' / f'u900_{date}.nc'
    command = ('cdo', '-P', '36', '-f', 'nc4', 'ap2pl,90000', f'-setpartabn,{name_table}', '-merge', f'{windfile}', f'{presfile}', f'{onelevelfile}')
    
    run_futures.append(run_cmd(command))
    
run_jobs = dask.persist(run_futures)
progress(run_jobs, notebook=False)

In [ ]:
run_futures = []
for windfile, presfile in [(wind_files[40], pres_files[40])]: # 
    
    print(windfile[-20:-7])
    date = windfile[-20:-7]

    onelevelfile = work_dir / 'U_Wind' / f'u900_{date}.nc'
    command = ('cdo', '-P', '36', '-f', 'nc4', 'ap2pl,90000', f'-setpartabn,{name_table}', '-merge', f'{windfile}', f'{presfile}', f'{onelevelfile}')
    
    run_futures.append(run_cmd(command))
    
run_jobs = dask.persist(run_futures)
progress(run_jobs, notebook=False)

In [ ]:
print('done.')